<a href="https://colab.research.google.com/github/Oleonn/DataMining/blob/main/Data_mining_INaturalist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup (should be executed on startup)

##Setup and connection to Google Drive

In [1]:
!pip install pyinaturalist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.0 MB/s eta 0:00:00


In [2]:
from pyinaturalist.v1.observations import get_observation_species_counts
import json
from pyinaturalist import (
    Observation,
    pprint,
    get_observations,
    get_observation_species_counts
)

from PIL import Image

import pandas as pd
pd.options.display.max_colwidth = 100  #Sert a augmenter la qte de caracteres affiches pour chaque string
import random

import csv
import requests
import os
import shutil

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


##Directory and parameters

In [4]:
directory = "/content/drive/MyDrive/Projet_mellifere/Donnees/"
sp_classes = ["Asclepias_syriaca", "Daucus_carota", "Eutrochium_maculatum", "Leucanthemum_vulgare", "Solidago_canadensis"]
originals = "originals"

#Create class folders if they don't already exist in directory
for name in sp_classes:
  #Create the species folder
  sp_classes_path = os.path.join(directory, name)
  if not os.path.exists(sp_classes_path):
    os.makedirs(sp_classes_path)
    print(f"folder {sp_classes_path} created.")
  #Create the originals subfolder
  sp_classes_originals = os.path.join(sp_classes_path, originals)
  if not os.path.exists(sp_classes_originals):
    os.makedirs(sp_classes_originals)
    print(f"folder {sp_classes_originals} created.")


Before executing the next cell, export the occurences from the GBIF website for the targeted species. Make sure to select "INaturalist (research)" in the search filters. Download the occurences in Archive Darwin Core format. Open the zip file(s) and extract the "multimedia.txt" of every target species into their matching folder (in the species folder that is, not in the "originals" one).

#Data mining

##Data mining

In [ ]:
wished_format = "small" #either thumb, small, medium, original or large"
wished_nb = 2000 #total number of pictures wished for every species

In [ ]:
for name in sp_classes:
  #Creation of a random sample (n = wished_nb) from the list of pictures in the "multimedia.txt" file
  sp_classes_path = os.path.join(directory, name)
  img_list_complete = pd.read_csv(sp_classes_path+"/multimedia.txt", dtype=str, sep="\t")["identifier"].tolist()
  img_list_used = random.sample(img_list_complete, wished_nb)
  for line in range(len(img_list_used)):
    img_list_used[line] = img_list_used[line].replace("original", wished_format)

  #Import of all sampled pictures
  sp_classes_originals = os.path.join(sp_classes_path, originals)
  broken_images = []
  count = 1
  for img in img_list_used:
    # We can split the file based on '/' and extract the last split within the Python list below:
    file_name = img.split('/')[-2]
    file_name = f"{sp_classes_originals}/{file_name}.jpeg"  # Update file extension to .jpeg
    if count == 1:
      print(f"Download of {wished_nb} files for {name} has started in {sp_classes_originals}")
    if count % 100 == 0 and count < wished_nb:
      print(f"File {count} out of {wished_nb} for {name} has been downloaded in {sp_classes_originals}")
    if count == wished_nb:
      print(f"All {wished_nb} files for {name} have been downloaded in {sp_classes_originals}")
    count = count + 1
    # Now let's send a request to the image URL:
    r = requests.get(img, stream=True)
    # We can check that the status code is 200 before doing anything else:
    if r.status_code == 200:
        # This command below will allow us to write the data to a file as binary:
        with open(file_name, 'wb') as f:
            for chunk in r.iter_content(1024):
                f.write(chunk)
    else:
        # We will write all of the images back to the broken_images list:
        broken_images.append(img)

  with open(f"{sp_classes_originals}/img_list_used_{name}.csv", "w", newline="") as f:
    for item in img_list_used:
      f.write(item + ",\n")
    print(f"Mission {name} complete.")

print("Mission complete. Good work, 007")

Download of 2000 files for Asclepias_syriaca has started in /content/drive/MyDrive/Projet_mellifere/Donnees/Asclepias_syriaca/originals
File 100 out of 2000 for Asclepias_syriaca has been downloaded in /content/drive/MyDrive/Projet_mellifere/Donnees/Asclepias_syriaca/originals
File 200 out of 2000 for Asclepias_syriaca has been downloaded in /content/drive/MyDrive/Projet_mellifere/Donnees/Asclepias_syriaca/originals
File 300 out of 2000 for Asclepias_syriaca has been downloaded in /content/drive/MyDrive/Projet_mellifere/Donnees/Asclepias_syriaca/originals
File 400 out of 2000 for Asclepias_syriaca has been downloaded in /content/drive/MyDrive/Projet_mellifere/Donnees/Asclepias_syriaca/originals
File 500 out of 2000 for Asclepias_syriaca has been downloaded in /content/drive/MyDrive/Projet_mellifere/Donnees/Asclepias_syriaca/originals
File 600 out of 2000 for Asclepias_syriaca has been downloaded in /content/drive/MyDrive/Projet_mellifere/Donnees/Asclepias_syriaca/originals
File 700 out

##Image size validation

In [ ]:
#Checking if every image has at least 128 pixels of width and height, and deletion of those that don't
def size_check():
  for name in sp_classes:
    sp_classes_path = os.path.join(directory, name)
    sp_classes_originals = os.path.join(sp_classes_path, originals)

    for filename in os.listdir(sp_classes_originals):
      if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        img_path = os.path.join(sp_classes_originals, filename)
        img = Image.open(img_path)
        width, height = img.size

        if width < 128 or height < 128:
          print(f"{img_path} is too small (w {width} x h {height}) and has been deleted")
          os.remove(img_path)

    print(f"{name} checked with no mo' undersized images to declare!")
  print("Mission complete! 10-4")


#Checking if images have been deleted, and download of additional pictures if it's the case
def redownload():
  for name in sp_classes:
    sp_classes_path = os.path.join(directory, name)
    sp_classes_originals = os.path.join(sp_classes_path, originals)

    img_list = []
    for filename in os.listdir(sp_classes_originals):
      if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        img_list.append(filename)
    print(f"Number of images for {name} : {len(img_list)} out of {wished_nb} expected")

    if len(img_list) < wished_nb: #If a redownload is necessary
      compensation = wished_nb - len(img_list)
      img_list_used_csv = f"{sp_classes_originals}/img_list_used_{name}.csv"
      #Reading the CSV file
      with open(img_list_used_csv, "r") as f:
        reader = csv.reader(f)
        used_items = [row[0] for row in reader]
        for line in range(len(used_items)): #Replace the wished format for the "original" format in the links of the used_items list. Makes comparison with the img_list_complete possible
          used_items[line] = used_items[line].replace(wished_format, "original")

      #Creation of a random sample (n = compensation) from the list of pictures (only the unused) in the "multimedia.txt" file
      img_list_complete = pd.read_csv(sp_classes_path+"/multimedia.txt", dtype=str, sep="\t")["identifier"].tolist()
      unused_items = [item for item in img_list_complete if item not in used_items] #Finding unused items
      #Additional selection
      new_items = random.sample(unused_items, compensation)
      for line in range(len(new_items)): #Replace the wished format for the "original" format in the links of the used_items list. Makes comparison with the img_list_complete possible
        new_items[line] = new_items[line].replace("original", wished_format)
      #Appending to CSV (updating the .csv file to inclued newly selected items)
      with open(img_list_used_csv, "a", newline="") as f:
        for item in new_items:
          f.write(item + ",\n")

      #Downloading the newly sampled images
      broken_images = []
      count = 1
      for img in new_items:
        # We can split the file based on '/' and extract the last split within the Python list below:
        file_name = img.split('/')[-2]
        file_name = f"{sp_classes_originals}/{file_name}.jpeg"  # Update file extension to .jpeg
        if count == 1:
          print(f"Download of {compensation} files for {name} has started in {sp_classes_originals}")
        if count % 10 == 0 and count < compensation:
          print(f"File {count} out of {compensation} for {name} has been downloaded in {sp_classes_originals}")
        if count == compensation:
          print(f"All {compensation} additional files for {name} have been downloaded in {sp_classes_originals}")
        count = count + 1
        # Now let's send a request to the image URL:
        r = requests.get(img, stream=True)
        # We can check that the status code is 200 before doing anything else:
        if r.status_code == 200:
            # This command below will allow us to write the data to a file as binary:
            with open(file_name, 'wb') as f:
                for chunk in r.iter_content(1024):
                    f.write(chunk)
        else:
            # We will write all of the images back to the broken_images list:
            broken_images.append(img)

Alternate between the two following cells to
1) check the size of every picture and delete those that are under 128pixels of width and height
2) redownload pictures to replace the deleted ones with new, newly sampled and so far unused pictures
Make sure the Google Drive has had time to synch (it may take up to 10 seconds) after each redownload

In [ ]:
size_check()

Asclepias_syriaca checked with no mo' undersized images to declare!
Daucus_carota checked with no mo' undersized images to declare!
Eutrochium_maculatum checked with no mo' undersized images to declare!
Leucanthemum_vulgare checked with no mo' undersized images to declare!
Solidago_canadensis checked with no mo' undersized images to declare!
Mission complete! 10-4


In [ ]:
redownload()

Number of images for Asclepias_syriaca : 2000 out of 2000 expected
Number of images for Daucus_carota : 2000 out of 2000 expected
Number of images for Eutrochium_maculatum : 2000 out of 2000 expected
Number of images for Leucanthemum_vulgare : 2000 out of 2000 expected
Number of images for Solidago_canadensis : 2000 out of 2000 expected


##Cropping

###Centered 128px square

In [ ]:
center_cropped = "center_cropped"

for name in sp_classes:
  #Create the center cropped subfolder
  sp_classes_path = os.path.join(directory, name)
  sp_classes_originals = os.path.join(sp_classes_path, originals)
  sp_classes_cropped = os.path.join(sp_classes_path, center_cropped)
  if not os.path.exists(sp_classes_cropped):
      os.makedirs(sp_classes_cropped)
      print(f"folder {sp_classes_cropped} created.")

  #Center crop
  print(f"Cropping of {name} images has started. Data will be saved in {sp_classes_cropped}")
  for filename in os.listdir(sp_classes_originals):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
      img_path = os.path.join(sp_classes_originals, filename)
      img = Image.open(img_path)
      width, height = img.size
      #Setting where to crop
      left = (width-128)/2 + ((width-128)/2)%1 #The modulo section here make it so that the answer is always a whole number
      right = left +128
      top = (height-128)/2 + ((height-128)/2)%1
      bottom = top +128
      #Crop and save in center_cropped folder
      img_cropped = img.crop((left,top,right,bottom))
      #Convert the image to RGB mode before saving as JPEG
      img_cropped = img_cropped.convert('RGB')
      file_name_cropped = f"{sp_classes_cropped}/{filename}"
      img_cropped.save(file_name_cropped)
  print(f"{name} images have been cropped to 128x128px and saved in {sp_classes_cropped}")

folder /content/drive/MyDrive/Projet_mellifere/Donnees/Asclepias_syriaca/center_cropped created.
Cropping of Asclepias_syriaca images has started. Data will be saved in /content/drive/MyDrive/Projet_mellifere/Donnees/Asclepias_syriaca/center_cropped
Asclepias_syriaca images have been cropped to 128x128px and saved in /content/drive/MyDrive/Projet_mellifere/Donnees/Asclepias_syriaca/center_cropped
folder /content/drive/MyDrive/Projet_mellifere/Donnees/Daucus_carota/center_cropped created.
Cropping of Daucus_carota images has started. Data will be saved in /content/drive/MyDrive/Projet_mellifere/Donnees/Daucus_carota/center_cropped
Daucus_carota images have been cropped to 128x128px and saved in /content/drive/MyDrive/Projet_mellifere/Donnees/Daucus_carota/center_cropped
folder /content/drive/MyDrive/Projet_mellifere/Donnees/Eutrochium_maculatum/center_cropped created.
Cropping of Eutrochium_maculatum images has started. Data will be saved in /content/drive/MyDrive/Projet_mellifere/Donne

###Resized so that shortest side=128 px, and cropped to 128x128





In [ ]:
resized_cropped = "resized_cropped"

for name in sp_classes:
  #Create the center cropped subfolder
  sp_classes_path = os.path.join(directory, name)
  sp_classes_originals = os.path.join(sp_classes_path, originals)
  sp_classes_cropped = os.path.join(sp_classes_path, resized_cropped)
  if not os.path.exists(sp_classes_cropped):
      os.makedirs(sp_classes_cropped)
      print(f"folder {sp_classes_cropped} created.")

  #Resizing and cropping
  print(f"Cropping of {name} images has started. Data will be saved in {sp_classes_cropped}")
  for filename in os.listdir(sp_classes_originals):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
      img_path = os.path.join(sp_classes_originals, filename)
      img = Image.open(img_path)
      width, height = img.size
      #Resize to shortest size = 128
      if width < height:
        ratio = 128/width
        new_width = 128
        new_height = int(height*ratio)
      else:
        ratio = 128/height
        new_width = int(width*ratio)
        new_height = 128
      resized_img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

      #Setting where to crop
      width, height = resized_img.size
      left = (width-128)/2 + ((width-128)/2)%1 #The modulo section here make it so that the answer is always a whole number
      right = left +128
      top = (height-128)/2 + ((height-128)/2)%1
      bottom = top +128
      #Crop and save in center_cropped folder
      img_cropped = resized_img.crop((left,top,right,bottom))
      #Convert the image to RGB mode before saving as JPEG
      img_cropped = img_cropped.convert('RGB')
      file_name_cropped = f"{sp_classes_cropped}/{filename}"
      img_cropped.save(file_name_cropped)
  print(f"{name} images have been cropped to 128x128px and saved in {sp_classes_cropped}")

folder /content/drive/MyDrive/Projet_mellifere/Donnees/Asclepias_syriaca/resized_cropped created.
Cropping of Asclepias_syriaca images has started. Data will be saved in /content/drive/MyDrive/Projet_mellifere/Donnees/Asclepias_syriaca/resized_cropped
Asclepias_syriaca images have been cropped to 128x128px and saved in /content/drive/MyDrive/Projet_mellifere/Donnees/Asclepias_syriaca/resized_cropped
folder /content/drive/MyDrive/Projet_mellifere/Donnees/Daucus_carota/resized_cropped created.
Cropping of Daucus_carota images has started. Data will be saved in /content/drive/MyDrive/Projet_mellifere/Donnees/Daucus_carota/resized_cropped
Daucus_carota images have been cropped to 128x128px and saved in /content/drive/MyDrive/Projet_mellifere/Donnees/Daucus_carota/resized_cropped
folder /content/drive/MyDrive/Projet_mellifere/Donnees/Eutrochium_maculatum/resized_cropped created.
Cropping of Eutrochium_maculatum images has started. Data will be saved in /content/drive/MyDrive/Projet_mellifer

#Data split

In [11]:
model_dir = "/content/drive/MyDrive/Projet_mellifere/Modeles/INat_resized-cropped"
data_folder = "resized_cropped"
train_data = 60
valid_data = 20
test_data = 20

if not os.path.exists(model_dir):
  os.makedirs(model_dir)
  print(f"folder {model_dir} created.")

Check if Google Drive has synced properly (that is, check if the newly created folder appears on the Drive)

In [12]:
#Create the target training, validation and test folders
train_path = f"{model_dir}/training_data"
if not os.path.exists(train_path):
  os.makedirs(train_path)
  print(f"folder {train_path} created.")

valid_path = f"{model_dir}/validation_data"
if not os.path.exists(valid_path):
  os.makedirs(valid_path)
  print(f"folder {valid_path} created.")

test_path = f"{model_dir}/test_data"
if not os.path.exists(test_path):
  os.makedirs(test_path)
  print(f"folder {test_path} created.")

Check if Google Drive has synced properly (that is, check if the newly created folder appears on the Drive)

In [13]:
for name in sp_classes:
  #Create the class folders inside train_path, valid_path and test_path
  train_path_sp = os.path.join(train_path, name)
  if not os.path.exists(train_path_sp):
    os.makedirs(train_path_sp)
    print(f"folder {train_path_sp} created.")

  valid_path_sp = os.path.join(valid_path, name)
  if not os.path.exists(valid_path_sp):
    os.makedirs(valid_path_sp)
    print(f"folder {valid_path_sp} created.")

  test_path_sp = os.path.join(test_path, name)
  if not os.path.exists(test_path_sp):
    os.makedirs(test_path_sp)
    print(f"folder {test_path_sp} created.")

Check if Google Drive has synced properly (that is, check if the newly created folder appears on the Drive)

In [14]:
def data_split(name, model_dir, data_folder, train_data, vali_data, test_data):

  #name : species_name as seen in sp_classes
  #model_dir : path to the model directory
  #data_folder : for every class, which folder to use (ex. resized_cropped)
  #train_data : percentage of training data (ex. 60)
  #valid_data : percentage of validation data (ex. 20)
  #test_data : percentage of test data (ex. 20)

  total = train_data + valid_data + test_data
  if total != 100:
    raise ValueError("The sum of training, validation and test percentages must be equal to 100")

  else:
  #Split the data in the right folders in the right proportions
    print(f"Proceeding with {name}")
    data_path = os.path.join(directory, name, data_folder)
    data_list = [f for f in os.listdir(data_path) if f.endswith(".jpeg")] #lists all .jpeg files in the data_path folder
    #Create the class folders inside train_path, valid_path and test_path
    train_path_sp = os.path.join(train_path, name)
    valid_path_sp = os.path.join(valid_path, name)
    test_path_sp = os.path.join(test_path, name)

    #Set up the proportions
    train_nb = int(train_data/100*len(data_list))
    if train_nb % 1 != 0 and train_nb % 1 < 0.5:
      train_nb = int(train_data/100*len(data_list) - train_data/100*len(data_list)%1)
    if train_nb %1 >= 0.5:
      train_nb = int(train_data/100*len(data_list) +1 - train_data/100*len(data_list)%1)

    valid_nb = int(valid_data/100*len(data_list))
    if valid_nb % 1 != 0 and valid_nb % 1 < 0.5:
      valid_nb = int(valid_data/100*len(data_list) - valid_data/100*len(data_list)%1)
    if valid_nb %1 >= 0.5:
      valid_nb = int(valid_data/100*len(data_list) +1 - valid_data/100*len(data_list)%1)

    test_nb = int(len(data_list) - train_nb - valid_nb)

    #Split the list
    random.shuffle(data_list)
    train_list = data_list[:train_nb]
    valid_list = data_list[train_nb: train_nb + valid_nb]
    test_list = data_list[train_nb + valid_nb:]

    #Copy train_list files and save them in train_path_sp
    for filename in train_list:
      file_path = os.path.join(data_path, filename)
      destination_path = os.path.join(train_path_sp, filename)
      shutil.copyfile(file_path, destination_path)
    print(f"{train_nb} files from {name} have been transfered to {train_path_sp}")

    #Copy valid_list files and save them in valid_path_sp
    for filename in valid_list:
      file_path = os.path.join(data_path, filename)
      destination_path = os.path.join(valid_path_sp, filename)
      shutil.copyfile(file_path, destination_path)
    print(f"{valid_nb} files from {name} have been transfered to {valid_path_sp}")

    #Copy test_list files and save them in test_path_sp
    for filename in test_list:
      file_path = os.path.join(data_path, filename)
      destination_path = os.path.join(test_path_sp, filename)
      shutil.copyfile(file_path, destination_path)
    print(f"{test_nb} files from {name} have been transfered to {test_path_sp}")

print("Mission complete. Good job, 007.")

Mission complete. Good job, 007.


Execute the following function one species at a time (by changing the selected item from sp_classes until it matches the length of sp_classes)
Leave enough time between every species for Google Drive to update and sync (the test_data folder of the previous species should be fully updated before executing further)

In [ ]:
name = sp_classes[4]
data_split(name, model_dir, data_folder, train_data, valid_data, test_data)